In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import re
# import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# from nltk.corpus import stopwords
from numpy import *

# Imports
# import pandas as pd
# from bs4 import BeautifulSoup
# import re
# from nltk.corpus import stopwords
# import nltk.data
# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
# from sklearn.feature_extraction.text import TfidfVectorizer


# initial imports
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# %matplotlib inline
# import matplotlib.pyplot as plt  # Matlab-style plotting
# import seaborn as sns
# color = sns.color_palette()
# sns.set_style('darkgrid')
# import warnings
# def ignore_warn(*args, **kwargs):
#     pass
# warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

# from scipy import stats
# from scipy.stats import norm, skew #for some statistics
# nltk.download('stopwords')
# from string import punctuation
# from nltk.corpus import stopwords
# from nltk import word_tokenize

In [2]:
all_data_with_sent = pd.read_csv('data/data_with_sentiment_score_and_one_hot.csv')
train_data_with_vecs = pd.read_csv('data/large_files/trainDataVecs.csv', header=None)
test_data_with_vecs = pd.read_csv('data/large_files/testDataVecs.csv', header=None)

In [3]:
all_data_with_vecs = pd.concat([train_data_with_vecs, test_data_with_vecs],ignore_index=True)

In [4]:
final_data = pd.concat([all_data_with_sent, all_data_with_vecs], axis=1)

In [ ]:
from html.parser import HTMLParser
parser = HTMLParser()

stop_words = stopwords.words('english') + list(punctuation)
stop_words = stop_words + ['–', '’', '“', '”']
def tokenize(text):
    text = parser.unescape(text)
    text = re.sub(' +', ' ',text.strip())
    text = re.sub("[^a-zA-Z]", " ", text)
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    return [w for w in words if w not in stop_words and not w.isdigit()]

In [ ]:
tfidf = TfidfVectorizer(stop_words=stop_words, tokenizer=tokenize, max_features=100)

In [ ]:
X = tfidf.fit_transform(final_data['text'])

In [20]:
import gc
gc.collect()

0

In [ ]:
X

In [ ]:
tfidf_values = pd.DataFrame(X.toarray(), columns=tfidf.get_feature_names())

In [6]:
c = list(final_data.columns)[9:]
c

['length_text',
 'Positive',
 'Negative',
 'Neutral',
 'Compound',
 'gender_female',
 'gender_male',
 'gender_nan',
 'sign_Aquarius',
 'sign_Aries',
 'sign_Cancer',
 'sign_Capricorn',
 'sign_Gemini',
 'sign_Leo',
 'sign_Libra',
 'sign_Pisces',
 'sign_Sagittarius',
 'sign_Scorpio',
 'sign_Taurus',
 'sign_Virgo',
 'sign_nan',
 'topic_Accounting',
 'topic_Advertising',
 'topic_Agriculture',
 'topic_Architecture',
 'topic_Arts',
 'topic_Automotive',
 'topic_Banking',
 'topic_Biotech',
 'topic_BusinessServices',
 'topic_Chemicals',
 'topic_Communications-Media',
 'topic_Construction',
 'topic_Consulting',
 'topic_Education',
 'topic_Engineering',
 'topic_Environment',
 'topic_Fashion',
 'topic_Government',
 'topic_HumanResources',
 'topic_Internet',
 'topic_InvestmentBanking',
 'topic_Law',
 'topic_LawEnforcement-Security',
 'topic_Manufacturing',
 'topic_Maritime',
 'topic_Marketing',
 'topic_Military',
 'topic_Museums-Libraries',
 'topic_Non-Profit',
 'topic_Publishing',
 'topic_RealEstat

In [ ]:
# tfidf_values.to_csv('tfidf_values_100.csv')

In [7]:
tfidf_values = pd.read_csv('tfidf_values.csv')

In [8]:
final_data_withtfidf = pd.concat([final_data[c], tfidf_values], axis=1)

In [9]:
n_train = len(train_data_with_vecs.index)

In [10]:
train_data = final_data_withtfidf[:n_train]

In [11]:
test_data = final_data_withtfidf[n_train:]

In [2]:
test = pd.read_csv('data/test.csv')

In [3]:
train = pd.read_csv('data/train.csv')

In [23]:
train_data.to_csv('train_data_vecs_tfidf.csv')
test_data.to_csv('test_data_vecs_tfidf.csv')

In [4]:
train_data = pd.read_csv('train_data_vecs_tfidf.csv')
test_data = pd.read_csv('test_data_vecs_tfidf.csv')

In [5]:
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)

In [7]:
c = list(train_data.columns)[1:]
train_data = train_data[c]
test_data = test_data[c]

In [8]:
# Initializing and fitting logistic regression model
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.svm import SVR

In [ ]:
lasso =  make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
mse_cv(lasso, train_data, train['age'])

In [ ]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1)
mse_cv(xgbm, train_data, train['age'])

In [ ]:
params = {'n_estimators': 360, 'max_depth': 2, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
gbr = GradientBoostingRegressor(**params)

mse_cv(gbr, train_data, train['age'])

In [21]:
data = final_data_withtfidf
# Checking other skewed features
numeric_feats = data.dtypes[data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(15)

MemoryError: 

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    data[feat] = boxcox1p(data[feat], lam)

In [ ]:
train_data = data[:n_train]
test_data = data[n_train:]
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)
train = train.fillna(0)

In [17]:
# n_folds = 2
def mse_cv(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    model.fit(X_train, y_train)
    true = y_test
    preds = model.predict(X_test)
    rmse= np.mean(np.square(np.absolute(preds - true)))
    return(rmse)

def mse_cv_log(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    model.fit(X_train, y_train)
    true = expm1(y_test)
    preds = expm1(model.predict(X_test))
    rmse= np.mean(np.square(np.absolute(preds - true)))
    return(rmse)

def mse_cv_neg_root(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    model.fit(X_train, y_train)
    true = np.power(y_test, -2)
    preds = np.power(model.predict(X_test), -2)
    rmse= np.mean(np.square(np.absolute(preds - true)))
    return(rmse)

def mse_cv_sin_norm(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#     kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    model.fit(X_train, y_train)
    true = np.power(y_test, -2)
    preds = np.power(model.predict(X_test), -2)
    rmse= np.mean(np.square(np.absolute(preds - true)))
    return(rmse)

In [ ]:
lasso =  make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
mse_cv(lasso, train_data, train['age'])

In [ ]:
def is_skewed(col):
    sns.distplot(train[col] , fit=norm);
    fig = plt.figure()
    res = stats.probplot(train[col], plot=plt)
    # Get the fitted parameters used by the function
    (mu, sigma) = norm.fit(train[col])
    print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

In [ ]:
train["age_t"] = np.log1p(train["age"])
is_skewed('age_t')

In [ ]:
lasso =  make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
mse_cv_log(lasso, train_data, train['age_t'])

In [ ]:
train["age_t2"] = np.power(train["age"], -0.5)
is_skewed('age_t2')

In [ ]:
lasso =  make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
mse_cv_neg_root(lasso, train_data, train['age_t2'])

In [ ]:
(max(train['age']) - min(train['age']))

In [ ]:
mean(train['age'])

In [ ]:
train["age_t3"] = np.sin((train["age"] - mean(train['age']))/(max(train['age']) - min(train['age'])))
train = train.fillna(0) 
is_skewed('age_t3')

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
mse_cv(KRR, train_data, train['age'])

In [10]:
xgbm = xgb.XGBRegressor(n_estimators=360, max_depth=2, learning_rate=0.1)
xgbm.fit(train_data, train['age'])
preds = xgbm.predict(test_data)
solution = pd.DataFrame({"user.id":test["user.id"], "age":preds})
solution = solution.groupby('user.id', as_index=False)['age'].mean()
solution.to_csv("solutions/word_vecs_tfidf_xgbm.csv", index = False)

In [9]:
train_data.head()

,length_text,Positive,Negative,Neutral,Compound,gender_female,gender_male,gender_nan,sign_Aquarius,sign_Aries,...,wrong,x,ya,yeah,year,years,yes,yesterday,yet,young
0,157,0.000000,0.000000,1.000000,0.000000,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,181,0.000000,0.000000,1.000000,0.000000,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2,25467,0.081101,0.070965,0.847934,0.003812,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.018583,0.0,0.017878,0.0
3,43,0.000000,0.000000,0.666667,0.000000,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
4,402,0.430500,0.000000,0.569500,0.393100,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [ ]:
test_data